##  Rozwiązywanie układu równań liniowych metodami rozkładu LU 
Podaj zasadę działania metod opartych o dekompozycję LU.

#### Zadanie 1
Zaimplementuj metody rozkładu LU:
- Rozkład Crouta 
- Rozkład Doolitla
- Rozkład Choleskyego 

Dla każdej z metod podaj warunki niezbędne aby można ją było zastosować. Sprawdź poprawność działania tych metod.  
Przetestuj wydajność algorytmów dla kilku rozmiarów macierzy (podobnie jak w ćwiczeniu 9).  

#### Odpowiedź:  
Dekompozycja LU (Lower-Upper) opiera się na rozbiciu macierzy wyjsciowej na dwie macierze trójkątne: L (macierz trójkątna dolna) i U (macierz trójkątna górna), takie że U * L = A, gdzie A to macierz wyjsciowa.
Dzięki takiej dekompozycji do zozwiązania układu możemy wykorzystać wzory na podstawianie wsteczne  
  
#### Warunki wykorzystania konkretnych metod:
   - Rozkład Crouta  
        - Na diagonali macierzy U występują same 1
   - Rozkład Doolitla  
        - Na diagonali macierzy L występują same 1
   - Rozkład Choleskyego   
        - Wartości na diagonalach macierzy L i U są sobie równe (u<sub>ii</sub> = l<sub>ii</sub>)
        - Macierz A jest symetryczna i dodatnio określona

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time

def crout_decomposition(A):
    n = len(A)
    L = [[0] * n for _ in range(n)]
    U = [[0] * n for _ in range(n)]
    for j in range(n):
        U[j][j] = 1

        for i in range(j, n):
            alpha = float(A[i][j])
            for k in range(j):
                alpha -= L[i][k]*U[k][j]
            L[i][j] = alpha

        for i in range(j+1, n):
            uu = float(A[j][i])
            for k in range(j):
                uu -= L[j][k]*U[k][i]

            if int(L[j][j]) == 0:
                raise ZeroDivisionError("0 occurred on diagonal. Could not compute U")

            U[j][i] = uu/L[j][j]

    return L, U


def doolittle_decomposition(A):
    n = len(A)
    L = [[0] * n for _ in range(n)]
    U = [[0] * n for _ in range(n)]

    for j in range(n):
        L[j][j] = 1
        for i in range(j + 1):
            s1 = 0
            for k in range(i):
                s1 += U[k][j] * L[i][k]
            U[i][j] = A[i][j] - s1
        for i in range(j, n):
            s2 = 0
            for k in range(j):
                s2 += U[k][j] * L[i][k]
            L[i][j] = (A[i][j] - s2) / U[j][j]

    return L, U



def cholesky_decomposition(A):
    n = len(A)
    L = [[0] * n for _ in range(n)]

    for i in range(n):
        for k in range(i + 1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            if i == k:
                L[i][k] = math.sqrt(A[i][i] - tmp_sum)
            else:
                L[i][k] = 1 / L[k][k] * (A[i][k] - tmp_sum)

    return L, np.array(L).transpose()  # L, U





def solve(A, b, L, U):
    z = np.linalg.solve(L, b)
    x = np.linalg.solve(U, z)
    return x


def generate_random_matrix(low, high, n):  # returns random symmetric and positive matrix
    A = np.random.randint(low, high, size=(n, n))
    return np.dot(A, A.transpose()), np.random.randint(low, high, size=(n, 1))


def compare(A, b):
    L1, U1 = crout_decomposition(A)
    L2, U2 = doolittle_decomposition(A)
    L3, U3 = cholesky_decomposition(A)
    
    print(f"\nCrout:\nL:")
    for i in range(len(L1)):
        print(L1[i])
    print(f"U:")
    
    for i in range(len(U1)):
        print(U1[i])
    print(f"\nDoolittle:\nL:")
    
    for i in range(len(L2)):
        print(L2[i])
    print(f"U:")
    for i in range(len(U2)):
        print(U2[i])

    print(f"\nCholesky:\nL:")
    for i in range(len(L3)):
        print(L3[i])
    print(f"U:")
    for i in range(len(U3)):
        print(U3[i])
    print("\n\nResult:\n")
    print(f"\tCrout ->\n {solve(A, b, L1, U1)}\n")
    print(f"\tDoolittle ->\n {solve(A, b, L2, U2)}\n")
    print(f"\tCholesky ->\n {solve(A, b, L3, U3)}")
    
    

In [2]:
A, b = generate_random_matrix(1, 100, 10)
compare(A, b)


Crout:
L:
[20580.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[17570.0, 16504.76190476191, 0, 0, 0, 0, 0, 0, 0, 0]
[16721.0, 5552.588435374151, 3902.3698343529572, 0, 0, 0, 0, 0, 0, 0]
[19176.0, 13583.653061224491, 3510.8739346361754, 4275.023921068574, 0, 0, 0, 0, 0, 0]
[21342.0, 9687.448979591838, 4522.7995461744995, 2310.016452679612, 5550.658747712926, 0, 0, 0, 0, 0]
[25742.0, 7857.986394557825, 2953.3270545356063, 1194.8839523188585, 2611.107453053908, 15216.651852008186, 0, 0, 0, 0]
[12751.0, 8130.942176870749, 2452.528025940036, 836.5285675132295, -1610.0377510970993, 8032.031319254981, 7794.351131242952, 0, 0, 0]
[22801.0, 10771.840136054423, -78.43481106550962, 7303.708982214771, 1734.5802927798945, 1544.3741052379082, 120.30995880601085, 2950.7334800851677, 0, 0]
[16655.0, 11723.935374149662, 2665.808393534276, 4078.8977230103146, -1834.7000341370808, 9451.186879751232, -2994.318838845653, 516.2403998264351, 824.3592501931378, 0]
[21369.0, 4709.397959183676, 1418.5953200919714, -345.352615

#### Zadanie 3
Zapoznaj się z funkcją rozwiązywania układów równań liniowych dostarczoną przez bibliotekę numpy/scipy. Porównaj jej wydajność z własnymi implementacjami.